# Information Theory

## Shamir's Secret Sharing

###### Odile, Gaspard, Antoine

## Outline

1. [Theory of SSS](#theory)
2. [Examples in low dimension.](#example)
3. [The code: a simulation of secret sharing.](#code)

In [1]:
import numpy as np

$\int$